In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import scala.util.{Failure, Success, Try}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession

VBox()

The code failed because of a fatal error:
	Session 55 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:41:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:41:56 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:41:57 INFO Configuration: resource-types.xml not found
24/03/15 06:41:57 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:41:57 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:41:57 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:41:57 INFO Client: Setting up container launch context for our AM
24/03/15 06:41:57 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [2]:
val spark = (
    SparkSession
    .builder()
    .appName("FinancialSheets")
    .config("spark.cores.max", 8)
    .config("spark.executor.cores", 8)
    .config("spark.executor.memory", "36g")
    .getOrCreate()
    )

VBox()

The code failed because of a fatal error:
	Session 51 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:24:39 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:24:39 INFO Configuration: resource-types.xml not found
24/03/15 06:24:39 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:24:39 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:24:39 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:24:39 INFO Client: Setting up container launch context for our AM
24/03/15 06:24:39 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [3]:
val cleanString = udf((str: String) => str.replaceAll("[^가-힣]", "").trim())
val replaceId = udf((sjDiv: String, accountId: String, accountNm: String) => sjDiv match {
    // 자산총계
    case "BS" if (accountId == "ifrs-full_Assets" || 
                  accountId == "ifrs_Assets" || 
                  (accountId == "not_defined" && accountNm == "자본총계")) => "Assets"
    // 유동자산 유동자산합계
    case "BS" if (accountId == "ifrs-full_CurrentAssets" || 
                  accountId == "ifrs_CurrentAssets" || 
                  (accountId == "not_defined" && accountNm == "유동자산") || 
                  (accountId == "not_defined" && accountNm == "유동자산합계")) => "CurrentAssets"
    // 현금및현금성자산
    case "BS" if (accountId == "ifrs-full_CashAndCashEquivalents" || 
                  accountId == "CashAndCashEquivalents" || 
                  (accountId == "not_defined" && accountNm == "현금및현금성자산")) => "CashAndCashEquivalents"
    // 매출채권
    case "BS" if (accountId == "dart_ShortTermTradeReceivable" || 
                  (accountId == "not_defined" && accountNm == "매출채권")) => "ShortTermTradeReceivable"
    // 비유동자산 비유동자산합계
    case "BS" if (accountId == "ifrs-full_NoncurrentAssets" || 
                  accountId == "ifrs_NoncurrentAssets" || 
                  (accountId == "not_defined" && accountNm == "비유동자산") || 
                  (accountId == "not_defined" && accountNm == "비유동자산합계")) => "NoncurrentAssets"
    // 유형자산 
    case "BS" if (accountId == "ifrs-full_PropertyPlantAndEquipment" || 
                  accountId == "ifrs_PropertyPlantAndEquipment" || 
                  (accountId == "not_defined" && accountNm == "유형자산")) => "PropertyPlantAndEquipment"
    // 무형자산
    case "BS" if (accountId == "ifrs-full_IntangibleAssetsOtherThanGoodwill" || 
                  accountId == "ifrs_IntangibleAssetsOtherThanGoodwill" || 
                  (accountId == "not_defined" && accountNm == "무형자산")) => "IntangibleAssetsOtherThanGoodwill"
    // 매출액
    case "IS" if (accountId == "ifrs-full_Revenue" || 
                  accountId == "ifrs_Revenue" || 
                  (accountId == "not_defined" && accountNm == "매출액") || 
                  (accountId == "not_defined" && accountNm == "수익")) => "Revenue"
    case "CIS" if (accountId == "ifrs-full_Revenue" || 
                   accountId == "ifrs_Revenue" || 
                   (accountId == "not_defined" && accountNm == "매출액") || 
                   (accountId == "not_defined" && accountNm == "수익")) => "CIS_Revenue"
    // 영업이익
    case "IS" if (accountId == "dart_OperatingIncomeLoss" || 
                  (accountId == "not_defined" && accountNm == "영업이익")) => "OperatingIncomeLoss"
    case "CIS" if (accountId == "dart_OperatingIncomeLoss" || 
                   (accountId == "not_defined" && accountNm == "영업이익")) => "CIS_OperatingIncomeLoss"
    // 당기순이익
    case "IS" if (accountId == "ifrs-full_ProfitLoss" || 
                  accountId == "ifrs_ProfitLoss" || 
                  (accountId == "not_defined" && accountNm == "당기순이익")) => "ProfitLoss"
    case "CIS" if (accountId == "ifrs-full_ProfitLoss" || 
                   accountId == "ifrs_ProfitLoss" || 
                   (accountId == "not_defined" && accountNm == "당기순이익")) => "CIS_ProfitLoss"
    // 영업활동현금흐름
    case "CF" if (accountId == "ifrs-full_CashFlowsFromUsedInOperatingActivities" || 
                  accountId == "ifrs_CashFlowsFromUsedInOperatingActivities" || 
                  (accountId == "not_defined" && accountNm == "영업활동현금흐름")) => "CashFlowsFromUsedInOperatingActivities"
    // 총부채
    case "BS" if (accountId == "ifrs-full_Liabilities" || 
                  accountId == "ifrs_Liabilities" || 
                  (accountId == "not_defined" && accountNm == "총부채") || 
                  (accountId == "not_defined" && accountNm == "부채총계")) => "Liabilities"
    // 매입채무
    case "BS" if (accountId == "dart_ShortTermTradePayables" || 
                  (accountId == "not_defined" && accountNm == "매입채무")) => "ShortTermTradePayables"
    // 자본총계
    case "BS" if (accountId == "ifrs-full_Equity" || 
                  accountId == "ifrs_Equity" || 
                  (accountId == "not_defined" && accountNm == "자본총계")) => "Equity"
    // 법인세비용차감전순이익
    case "IS" if (accountId == "ifrs-full_ProfitLossBeforeTax" || 
                  accountId == "ifrs_ProfitLossBeforeTax" || 
                  (accountId == "not_defined" && accountNm == "법인세비용차감전순이익")) => "ProfitLossBeforeTax"
    case "CIS" if (accountId == "ifrs-full_ProfitLossBeforeTax" || 
                   accountId == "ifrs_ProfitLossBeforeTax" || 
                   (accountId == "not_defined" && accountNm == "법인세비용차감전순이익")) => "CIS_ProfitLossBeforeTax"
    // 자본금
    case "BS" if (accountId == "ifrs_IssuedCapital" || 
                  accountId == "ifrs-full_IssuedCapital" || 
                  (accountId == "not_defined" && accountNm == "자본금")) => "IssuedCapital"
    // 이익잉여금
    case "BS" if (accountId == "ifrs-full_RetainedEarnings" || 
                  accountId == "ifrs_RetainedEarnings" || 
                  (accountId == "not_defined" && accountNm == "이익잉여금")) => "RetainedEarnings"
    // 이자비용???
    case "CF" if (accountId == "dart_AdjustmentsForInterestExpenses" || 
                  (accountId == "not_defined" && accountNm == "이자비용")) => "AdjustmentsForInterestExpenses"
    // 재고자산
    case "BS" if (accountId == "ifrs-full_Inventories" || 
                  accountId == "ifrs_Inventories" || 
                  (accountId == "not_defined" && accountNm == "재고자산")) => "Inventories"
    // 매출원가
    case "IS" if (accountId == "ifrs-full_CostOfSales" || 
                  accountId == "ifrs_CostOfSales" || 
                  (accountId == "not_defined" && accountNm == "매출원가")) => "CostOfSales"
    case "CIS" if (accountId == "ifrs-full_CostOfSales" || 
                   accountId == "ifrs_CostOfSales" || 
                   (accountId == "not_defined" && accountNm == "매출원가")) => "CIS_CostOfSales"
    // 기타포괄손익
    case "CIS" if (accountId == "ifrs-full_OtherComprehensiveIncome" || 
                   accountId == "ifrs_OtherComprehensiveIncome" || 
                   (accountId == "not_defined" && accountNm == "기타포괄손익")) => "OtherComprehensiveIncome"
    // 유동부채
    case "BS" if (accountId == "ifrs-full_CurrentLiabilities" || 
                  accountId == "ifrs_CurrentLiabilities" || 
                  (accountId == "not_defined" && accountNm == "유동부채")) => "CurrentLiabilities"
    // 이자지급(영업,투자,재무)
    case "CF" if (accountId == "ifrs-full_InterestPaidClassifiedAsFinancingActivities" || 
                  accountId == "ifrs-full_InterestPaidClassifiedAsOperatingActivities" || 
                  accountId == "ifrs-full_InterestPaidClassifiedAsInvestingActivities" || 
                  accountId == "ifrs_InterestPaidClassifiedAsFinancingActivities" || 
                  accountId == "ifrs_InterestPaidClassifiedAsOperatingActivities" || 
                  accountId == "ifrs_InterestPaidClassifiedAsInvestingActivities" || 
                  (accountId == "not_defined" && accountNm == "이자지급")) => "InterestPaid"
    // 비지배지분
    case "BS" if (accountId == "ifrs-full_NoncontrollingInterests" || 
                  accountId == "ifrs_NoncontrollingInterests" || 
                  (accountId == "not_defined" && accountNm == "비지배지분")) => "NoncontrollingInterests"
    // 감가상각비
    case "BS" if (accountId == "dart_AccumulatedDepreciationBuildingsGross" || 
                  accountId == "dart_AccumulatedDepreciationBuildingIncidentalFacilitiesGross" || 
                  accountId == "dart_AccumulatedDepreciationFittingGross" || 
                  accountId == "dart_AccumulatedDepreciationStructureGross" || 
                  accountId == "dart_AccumulatedDepreciationMachineryGross" || 
                  accountId == "dart_AccumulatedDepreciationVehiclesGross" || 
                  accountId == "dart_AccumulatedDepreciationSupplyFacilitiesGross" || 
                  accountId == "dart_AccumulatedDepreciationElectronicFacilitiesGross" || 
                  accountId == "dart_AccumulatedDepreciationToolsAndEquipmentGross" || 
                  accountId == "dart_AccumulatedDepreciationFinanceLeaseAssetGross" || 
                  accountId == "dart_AccumulatedDepreciationRentStructureGross" || 
                  accountId == "dart_AccumulatedDepreciationFixturesAndFittingsGross" || 
                  accountId == "dart_AccumulatedDepreciationOfficeEquipmentGross" || 
                  accountId == "dart_AccumulatedDepreciationTangibleExplorationAndEvaluationAssetsGross" || 
                  accountId == "dart_AccumulatedDepreciationRentalAssetGross" || 
                  accountId == "dart_AccumulatedDepreciationShipsGross" || 
                  accountId == "dart_AccumulatedDepreciationOtherPropertyPlantAndEquipmentGross" || 
                  (accountId == "not_defined" && accountNm == "감가상각비")) => "AccumulatedDepreciation"
    // 대손충당금
    case "BS" if (accountId == "dart_AllowanceForDoubtfulAcccountShortTermTradeReceivable" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountCurrentFinanceLeaseReceivables" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermDeferredAncillaryIncomeForLoans" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermOtherReceivables" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountReceivablesOnConstructionContracts" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountReceivablesRealestateSales" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountReceivablesAgent" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermAccruedIncome" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermDepositsProvided" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermAdvancePayments" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermPrepaidExpenses" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermPrepaidConstructionCosts" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLeaseholdDeposits" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermDueFromCustomersForContractWork" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountShortTermLoans" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermTradeReceivablesGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountNoncurrentFinanceLeaseReceivable" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermsIncomeForLoans" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermOtherReceivablesGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermReceivablesOnConstructionContracts" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermReceivablesRealestateSales" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermReceivablesAgent" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermAccruedIncomeGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermDepositsProvidedGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermAdvancePaymentsGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermPrepaidExpensesGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermLoansGross" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermLeaseholdDeposits" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermDueFromCustomersForContractWork" || 
                  accountId == "dart_AllowanceForDoubtfulAcccountLongTermContractReserve" || 
                  (accountId == "not_defined" && accountNm == "대손충당금")) => "AllowanceForDoubtfulAcccount"

//hanseong 230626 start

    // 미지급금
    case "BS" if (accountId =="dart-full_ShortTermOtherPayables" ||
                  accountId =="dart_ShortTermOtherPayables" ||
                  (accountId == "not_defined" && accountNm == "미지급금")) =>"ShortTermOtherPayables"
    // 차입금    
    case "BS" if (accountId =="ifrs-full_ShorttermBorrowings" || 
                  accountId =="ifrs_ShorttermBorrowings" || 
                  (accountId == "not_defined" && accountNm == "차입금")) =>"ShorttermBorrowings" 
    // 금융수익   
    case "IS" if (accountId =="ifrs-full_FinanceIncome" ||
                  accountId =="ifrs_FinanceIncome" || 
                  (accountId == "not_defined" && accountNm == "금융수익")) =>"FinanceIncome" 
    case "CIS" if (accountId =="ifrs-full_FinanceIncome" ||
                  accountId =="ifrs_FinanceIncome" || 
                  (accountId == "not_defined" && accountNm == "금융수익")) =>"CIS_FinanceIncome" 
    // 금융원가    
    case "IS" if (accountId =="ifrs-full_FinanceCosts" ||
                  accountId =="ifrs_FinanceCosts" ||
                  (accountId == "not_defined" && accountNm == "금융원가")) =>"FinanceCosts" 
    case "CIS" if (accountId =="ifrs-full_FinanceCosts" ||
                  accountId =="ifrs_FinanceCosts" ||
                  (accountId == "not_defined" && accountNm == "금융원가")) =>"CIS_FinanceCosts" 
    // 판매비와관리비
    case "IS" if (accountId =="dart-full_TotalSellingGeneralAdministrativeExpenses" ||
                  accountId =="dart_TotalSellingGeneralAdministrativeExpenses" ||
                  (accountId == "not_defined" && accountNm == "판매비와관리비")) =>"TotalSellingGeneralAdministrativeExpenses" 
    case "CIS" if (accountId =="dart-full_TotalSellingGeneralAdministrativeExpenses" ||
                   accountId =="dart_TotalSellingGeneralAdministrativeExpenses" ||
                   (accountId == "not_defined" && accountNm == "판매비와관리비")) =>"CIS_TotalSellingGeneralAdministrativeExpenses" 
    // 차입금의 상환    
    case "CF" if (accountId =="ifrs-full_RepaymentsOfBorrowingsClassifiedAsFinancingActivities" ||
                   accountId =="ifrs_RepaymentsOfBorrowingsClassifiedAsFinancingActivities" ||
                   (accountId == "not_defined" && accountNm == "차입금의 상환")) =>"RepaymentsOfBorrowingsClassifiedAsFinancingActivities" 
    // 유형자산의 취득
    case "CF" if (accountId =="ifrs-full_PurchaseOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities" ||
                  accountId =="ifrs_PurchaseOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities" ||
                  (accountId == "not_defined" && accountNm == "유형자산의 취득")) =>"PurchaseOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities" 
    // 유형자산의 처분
    case "CF" if (accountId =="ifrs-full_ProceedsFromSalesOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities" ||
                  accountId =="ifrs_ProceedsFromSalesOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities" ||
                  (accountId == "not_defined" && accountNm == "유형자산의 처분")) =>"ProceedsFromSalesOfPropertyPlantAndEquipmentClassifiedAsInvestingActivities" 
    // 유동충당부채   
    case "BS" if (accountId =="ifrs-full_CurrentProvisions" ||
                  accountId =="ifrs_CurrentProvisions" ||
                  (accountId == "not_defined" && accountNm == "유동충당부채")) =>"CurrentProvisions" 
    
//hanseong 230626 end
    
    case _ => ""
    }
)

VBox()

The code failed because of a fatal error:
	Session 51 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:24:39 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:24:39 INFO Configuration: resource-types.xml not found
24/03/15 06:24:39 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:24:39 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:24:39 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:24:39 INFO Client: Setting up container launch context for our AM
24/03/15 06:24:39 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [ ]:
val mongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

VBox()

The code failed because of a fatal error:
	Session 51 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:24:39 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:24:39 INFO Configuration: resource-types.xml not found
24/03/15 06:24:39 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:24:39 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:24:39 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:24:39 INFO Client: Setting up container launch context for our AM
24/03/15 06:24:39 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [5]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2015/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )
    
//     .where(col("account_id")==="ifrs_RepaymentsOfBorrowingsClassifiedAsFinancingActivities")
//     .select("stock_name","account_id",)
//     )

VBox()

The code failed because of a fatal error:
	Session 51 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:24:39 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:24:39 INFO Configuration: resource-types.xml not found
24/03/15 06:24:39 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:24:39 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:24:39 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:24:39 INFO Client: Setting up container launch context for our AM
24/03/15 06:24:39 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [ ]:
ndf = df.filter(df["account_id"] == "assets")
ndf.show()

In [ ]:
"""
(
    df
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [13]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2016/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [ ]:
"""
(
    df
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [ ]:
val df = (
    spark.read
    .option("header",true)
//     .option("multiLine",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2017/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
"""
(
    df
//     .printSchema()
//     .show(10000)
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
"""

In [ ]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2018/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
"""
(
    df
//     .printSchema()
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [ ]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2019/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
"""
(
    df
//     .printSchema()
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [ ]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2020/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
"""
(
    df
//     .printSchema()
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [ ]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2021/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
"""
(
    df
//     .printSchema()
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [ ]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2022/*/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
"""
(
    df
//     .printSchema()
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [ ]:
val df = (
    spark.read
    .option("header",true)
    .csv("s3a://domestic-original-raw/dart/financial_sheet/2023/08/*.csv")
    .withColumn("updateDate", substring(col("rcept_no"), 1, 8))
    .withColumn("thstrm_amount", col("thstrm_amount").cast(LongType))
    .withColumn("account_id", regexp_replace(col("account_id"), "-표준계정코드 미사용-", "not_defined"))
    .withColumn("account_nm", regexp_replace(col("account_nm"), " ", ""))
    .withColumn("account_nm", cleanString(col("account_nm")))
    .withColumn("account_id", replaceId(col("sj_div"), col("account_id"), col("account_nm")))
    .where(col("account_id") !== "")
    .withColumnRenamed("rcept_no", "rceptNo")
    .withColumnRenamed("reprt_code", "reprtCode")
    .withColumnRenamed("bsns_year", "bsnsYear")
    .withColumnRenamed("corp_code", "corpCode")
    .withColumnRenamed("sj_div", "sjDiv")
    .withColumnRenamed("sj_nm", "sjNm")
    .withColumnRenamed("account_id", "accountId")
    .withColumnRenamed("account_nm", "accountNm")
    .withColumnRenamed("account_detail", "accountDetail")
    .withColumnRenamed("thstrm_nm", "thstrmNm")
    .withColumnRenamed("thstrm_amount", "thstrmAmount")
    .withColumnRenamed("stock_name", "stockName")
    .withColumnRenamed("stock_code", "stockCode")
    .withColumnRenamed("fs_div", "fsDiv")
    )

In [ ]:
df.printSchema()

In [ ]:
"""
(
    df
//     .printSchema()
    .write.format("mongodb")
    .mode("append")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .save()
    )
    """

In [6]:
val df = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportCleansing")
    .load()
    .drop("_id")
    )

VBox()

The code failed because of a fatal error:
	Session 51 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:24:39 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:24:39 INFO Configuration: resource-types.xml not found
24/03/15 06:24:39 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:24:39 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:24:39 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:24:39 INFO Client: Setting up container launch context for our AM
24/03/15 06:24:39 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [7]:
val ndf = df.filter((df("accountId") === "assets") && (df("accountNm") === "자본총계"))
ndf.show()

VBox()

The code failed because of a fatal error:
	Session 51 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 06:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 06:24:39 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 06:24:39 INFO Configuration: resource-types.xml not found
24/03/15 06:24:39 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 06:24:39 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 06:24:39 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 06:24:39 INFO Client: Setting up container launch context for our AM
24/03/15 06:24:39 INFO Client: Setting up the launch environment for our AM container
24/03/15 06:

In [ ]:
spark.stop()